In [25]:
import numpy as np
import cv2, os

## Load Ground Truth Points

In [26]:
directory = './data/xview'
region_name = '322_car_1'
grid_size = 20

gt_file = os.path.join(directory, region_name+'_gt.txt')
gt_points = np.loadtxt(gt_file, dtype='int32', delimiter=',')
print(gt_points.shape)

(274, 2)


## Load Predicted Points

In [27]:
pred_file = os.path.join(directory, region_name+'_points_pred.txt')
pred_points = np.loadtxt(pred_file, dtype='int32', delimiter=',')
print(pred_points.shape)

(850, 2)


## Generate True Pos & True Neg Grids

In [28]:
offset = grid_size # the gt point is in the grid

mask = cv2.imread(os.path.join(directory, region_name+'_label.jpeg'),0)
# mask = cv2.imread('./data/xview/322_car_1_label.jpeg', 0)
true_pos_grids = []
true_neg_grids = []

for row in range(0,mask.shape[0]-grid_size,grid_size):
    for col in range(0,mask.shape[1]-grid_size,grid_size):
        grid_x = row 
        grid_y = col 
        flag = 0
        for x, y in gt_points:
            if x-grid_x<=offset and x-grid_x>0 and y-grid_y<=offset and y-grid_y>0:
                true_pos_grids.append([grid_x, grid_y])
                flag = 1
                break
        if flag == 0:
            true_neg_grids.append([grid_x, grid_y])

# true_pos_grids, true_neg_grids = np.array(true_pos_grids), np.array(true_neg_grids)
print(len(true_pos_grids), len(true_neg_grids))

193 36221


In [29]:
# check true grids map

true_grids_map = np.zeros(mask.shape)
for i in true_pos_grids:
    x, y = i[0], i[1]
    true_grids_map[x:x+grid_size, y:y+grid_size] = 255
cv2.imwrite(region_name+'_true_grids_'+str(grid_size)+'.jpeg', true_grids_map)

True

## Generate Recognized Pos & Neg Grids

In [30]:

reg_pos_grids = []
reg_neg_grids = []

for row in range(0,mask.shape[0]-grid_size,grid_size):
    for col in range(0,mask.shape[1]-grid_size,grid_size):
        grid_x = row 
        grid_y = col 
        flag = 0
        for x, y in pred_points:
            if x-grid_x<=offset and x-grid_x>0 and y-grid_y<=offset and y-grid_y>0:
                reg_pos_grids.append([grid_x, grid_y])
                flag = 1
                break
        if flag == 0:
            reg_neg_grids.append([grid_x, grid_y])
        

# reg_pos_grids, reg_neg_grids = np.array(reg_pos_grids), np.array(reg_neg_grids)
print(len(reg_pos_grids), len(reg_neg_grids))

414 36000


In [31]:
# check recognized grids map

reg_grids_map = np.zeros(mask.shape)
for i in reg_pos_grids:
    x, y = i[0], i[1]
    reg_grids_map[x:x+grid_size, y:y+grid_size] = 255
cv2.imwrite(region_name+'_pred_grids_'+str(grid_size)+'.jpeg', reg_grids_map)

True

## Calculate Precision, Recall, F1

In [32]:
tp, fp, fn = 0.0, 0.0, 0.0

for i in reg_pos_grids:
    if i in true_pos_grids:
#         print(i, true_pos_grids.where(i))
        tp += 1
    else:
        fp += 1

for i in true_pos_grids:
    if i not in reg_pos_grids:
        fn += 1

precision = tp / (tp+fp)
recall = tp / (tp+fn)
f1 = 2*precision*recall / (precision+recall)
print(tp, fp, fn)
print(precision, recall, f1)

174.0 240.0 19.0
0.42028985507246375 0.9015544041450777 0.5733113673805601
